# Proyecto 3

**Tema:** Resultados Saber 11



|     Nombres      |      Apellidos       |     Login     |  Codigo   |
| :--------------: | :------------------: | :-----------: | :-------: |
|     Santiago     | Gonzalez Montealegre | s.gonzalez35  | 202012274 |
| Juliana Carolina |  Cardenas Barragan   | jc.cardenasb1 | 202011683 |


## Librerías Requeridas

In [1]:
import pandas as pd

## Carga de Datos

In [2]:
original = pd.read_csv('../../Data/Transformed/Clean.csv')

In [14]:
original.head()

,cole_area_ubicacion,cole_bilingue,cole_calendario,cole_naturaleza,cole_genero,cole_jornada,cole_cod_depto_ubicacion,cole_cod_mcpio_ubicacion,estu_cod_depto_presentacion,estu_cod_mcpio_presentacion,estu_cod_reside_depto,estu_cod_reside_mcpio,estu_genero,fami_estratovivienda,fami_educacionmadre,fami_educacionpadre,fami_personashogar,fami_tienecomputador,fami_tieneinternet,punt_global
0,URBANO,N,A,OFICIAL,MIXTO,MAÑANA,13,13001,13,13001,13,13001,F,Estrato 1,Secundaria (Bachillerato) incompleta,Primaria incompleta,Siete,Si,Si,205
1,URBANO,N,A,OFICIAL,MIXTO,MAÑANA,70,70523,70,70001,70,70523,F,Estrato 1,Primaria completa,Primaria completa,5 a 6,No,No,187
2,URBANO,N,A,OFICIAL,MIXTO,MAÑANA,70,70523,70,70001,70,70523,F,Estrato 1,Primaria completa,Primaria completa,5 a 6,No,No,187
3,RURAL,N,A,NO OFICIAL,MASCULINO,COMPLETA,11,11001,11,11001,11,11001,F,Estrato 4,Educación profesional completa,Postgrado,Cuatro,Si,Si,313
4,URBANO,N,A,OFICIAL,MIXTO,MAÑANA,25,25899,25,25899,25,25899,F,Estrato 1,Primaria incompleta,Primaria incompleta,7 a 8,Si,Si,201


## Transformación

In [40]:
df = original.groupby(['cole_area_ubicacion', 'cole_naturaleza']).agg(punt_promedio = ('punt_global', 'mean')).reset_index()

df.loc[df['cole_area_ubicacion'] == 'RURAL', 'cole_area_ubicacion'] = 'Rural'
df.loc[df['cole_area_ubicacion'] == 'URBANO', 'cole_area_ubicacion'] = 'Urbano'

df.loc[df['cole_naturaleza'] == 'NO OFICIAL', 'cole_naturaleza'] = 'Privado'
df.loc[df['cole_naturaleza'] == 'OFICIAL', 'cole_naturaleza'] = 'Publico'

In [41]:
df.head()

,cole_area_ubicacion,cole_naturaleza,punt_promedio
0,Rural,Privado,302.447131
1,Rural,Publico,232.524394
2,Urbano,Privado,277.038807
3,Urbano,Publico,250.853328


## Gráfica

In [4]:
import plotly.express as px

In [42]:
fig = px.bar(
    data_frame=df,
    y='cole_area_ubicacion',
    x='punt_promedio',
    color='cole_naturaleza',
    orientation='h',
    barmode='group'
    
)
fig.update_layout(
    xaxis_range=[0, 500],
    legend_title_text='Tipo'
)
fig.update_xaxes(
    title='Puntaje Promedio'
)
fig.update_yaxes(
    title='Zona Ubicación'
)
fig.show()

## Creación Script Base de Datos

In [43]:
sql_script = """
CREATE TABLE promedio_zona (
    zona VARCHAR(255),
    tipo VARCHAR(255),
    punt_promedio NUMBER
);
"""

In [44]:
sql_script += 'INSERT INTO promedio_zona (zona, tipo, punt_promedio) VALUES\n'

values = []
for i, row in df.iterrows():
    value = f"('{row['cole_area_ubicacion']}', '{row['cole_naturaleza']}', {float(row['punt_promedio'])})"
    # value = f"({int(row['age'])}, {int(row['sex'])}, {int(row['cp'])}, {int(row['trestbps'])}, {int(row['chol'])}, {int(row['fbs'])}, {int(row['restecg'])}, {int(row['thalach'])}, {int(row['exang'])}, {row['oldpeak']}, {int(row['slope'])}, {int(row['ca'])}, {int(row['thal'])}, {int(row['heartdis'])})"
    values.append(value)
    
sql_script += ',\n'.join(values) + ';'

## Guardar Script

In [45]:
with open("../../Data/Scripts/Zona_Script.sql", "w") as sql_file:
    sql_file.write(sql_script)